In [3]:
import pandas as pd
import numpy as np
import datetime
import time
import random
from kafka import KafkaConsumer, KafkaProducer
from json import loads, dumps
from collections import Counter

import sys
sys.path.append('./CountMinSketch/')

from countminsketch import CountMinSketch
# from hashfactory import hash_function

In [4]:
seed=0
np.random.seed(seed)
random.seed(seed)

In [5]:
file = pd.read_csv('november_2021_COVID-19_Twitter_Streaming_Dataset.csv')

In [7]:
def find_tweet_timestamp_post_snowflake(tid):
    offset = 1288834974657
    tstamp = (tid >> 22) + offset
    return tstamp

In [8]:
import random
import hashlib

_memomask = {}
def hash_function(n):
    """
    :param n: the index of the hash function
    :return: a generated hash function
    """
    mask = _memomask.get(n)
    bit=4
    if mask is None:
        random.seed(n)
        mask = _memomask[n] = random.getrandbits(bit)
        # print(mask)

    def my_hash(x):
        # print('x' ,x)
        # print(str(x) + str(n))
        return hash(str(x) + str(n)) ^ mask

    return my_hash

In [9]:
DEPTH = 7
# WIDTH = 2**22
WIDTH = 5
HASH_FUNCTIONS = [hash_function(i) for i in range(DEPTH)]

In [10]:
hash_map = []
for i in range(len(HASH_FUNCTIONS)):
    list_=[]
    for j in range((WIDTH)):
        list_.append(HASH_FUNCTIONS[i](j)%WIDTH)
        print(HASH_FUNCTIONS[i](j)%WIDTH, end=' ')
    hash_map.append(list_)
    print()

1 2 4 1 4 
3 1 1 4 1 
4 3 1 4 4 
4 4 1 4 0 
2 3 1 2 3 
0 1 1 3 0 
0 1 0 3 1 


In [11]:
hash_map =np.array(hash_map).T

In [12]:
batch = Counter()
sketch = CountMinSketch(DEPTH, WIDTH, HASH_FUNCTIONS)
for idx, tid in enumerate(file['_id']):
    # print(tid)
    timestamp = find_tweet_timestamp_post_snowflake(tid)
    current_date = datetime.datetime.fromtimestamp(timestamp/1000)
    # print(current_date.hour, current_date.ctime().split()[0])
    batch[current_date.hour]+=1
    for key, count in batch.items():
        # print(key, count)
        sketch.add(key, count)
    batch.clear()
    
    # if idx==100:
    #     break

KeyboardInterrupt: 

In [27]:
idx

6

In [13]:
sketch.get_matrix()

array([[3469601, 3970395, 3996707, 1481852, 3334836],
       [1829975, 5246030, 2781336, 1962472, 4433578],
       [1273583, 1377384, 3076463, 3014971, 7510990],
       [3201415, 2529839, 2568852, 4986012, 2967273],
       [3782249, 1175518, 4694965, 3379589, 3221070],
       [3112963, 5080318, 3398419, 1983244, 2678447],
       [6026014, 2781726, 1897374, 2421347, 3126930]], dtype=int32)

In [15]:
hash_map[0]

array([1, 3, 4, 4, 2, 0, 0])

In [26]:
min([sketch.get_matrix()[idx][j] for idx, j in enumerate(hash_map[0])])

1962472